In [1]:
import torch
import sys
sys.path.append('/home/amber/multitask_RNA/rna_self_train/')
import os
os.environ['CUDA_VISIBLE_DEVICES']='7'
import rna_model
import h5py
import numpy as np
from tqdm import tqdm

In [2]:
config_dict = {
    'model':'conv_former',
    'hidden_size': 512,
    'attention_window': [256,256,256],
    'num_attention_heads': 8,
    'intermediate_size': 2048,
    'attention_dilation': [1,1,1],
    'data_dir' : '/grid/koo/home/ztang/multitask_RNA/data/pre-train/3072/rna_onehot.h5',
    'batch_size':16}
ckpt_path = '/home/amber/multitask_RNA/rna_self_train/model_ckpt/convformer_linear_decay/epoch=73-step=23200.ckpt'

In [3]:
config = rna_model.conv_former_config(**config_dict)
model = rna_model.conv_former.load_from_checkpoint(ckpt_path,config = config).cuda()
model.eval()

conv_former(
  (conv1): Conv1d(4, 512, kernel_size=(19,), stride=(1,), padding=same)
  (batchnorm): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (block1): dilated_residual(
    (block): Sequential(
      (0): Conv1d(512, 512, kernel_size=(7,), stride=(1,), padding=same)
      (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.1, inplace=False)
      (4): Conv1d(512, 512, kernel_size=(7,), stride=(1,), padding=same, dilation=(2,))
      (5): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (output_act): ReLU()
  )
  (block2): dilated_residual(
    (block): Sequential(
      (0): Conv1d(512, 512, kernel_size=(7,), stride=(1,), padding=same)
      (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.1, inplace=False)
      (4): Conv1d(512, 512, kernel_size=(7,), 

In [4]:
activation = {}
def get_activation(name):
    def hook(model,input,output):
        if name not in activation.keys():
            activation[name] = []
        activation[name].extend(output[0].cpu().detach().numpy())
    return hook

model.att_list[-1].register_forward_hook(get_activation('att'))

In [5]:
file = h5py.File('../../data/rna_stable/insert_dataset.h5','r')
convformer_output = h5py.File('../../data/rna_stable/convformer_output.h5','w')
batch_size = 16
for dataset in ['test','train','valid']:
    print(dataset)
    key = 'X_'+dataset
    onehot = np.transpose(file[key][()],(0,2,1))
    pad_onehot = np.pad(onehot,((0,0),(0,0),(169,170)), mode='constant', constant_values=0)
    print(pad_onehot.shape)

    for seq_i in tqdm(range(0,len(pad_onehot),batch_size)):
        seq_batch = torch.tensor(pad_onehot[seq_i:seq_i+batch_size]).to('cuda')
        output = model(torch.tensor(seq_batch))
        
    convformer_rep = np.array(activation['att'])[:,169:-170,:]
    print(convformer_rep.shape)
    convformer_output.create_dataset(key, data=convformer_rep)
    convformer_output.create_dataset(name='Y_'+dataset,data = file['Y_'+dataset][:])
    activation = {}

convformer_output.close()

test
(1137, 4, 512)


  0%|          | 0/72 [00:00<?, ?it/s]/tmp/ipykernel_1365600/3172365481.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = model(torch.tensor(seq_batch))
100%|██████████| 72/72 [00:03<00:00, 21.51it/s]


(1137, 173, 512)
train
(9131, 4, 512)


100%|██████████| 571/571 [00:26<00:00, 21.66it/s]


(9131, 173, 512)
valid
(1149, 4, 512)


100%|██████████| 72/72 [00:03<00:00, 23.00it/s]


(1149, 173, 512)


In [6]:
gpn_file = h5py.File('../../data/rna_stable/gpn_finetune_embed.h5','r')
conv_file = h5py.File('../../data/rna_stable/convformer_embed.h5','r')

In [7]:
gpn_file.keys()

<KeysViewHDF5 ['X_test', 'X_train', 'X_valid', 'Y_test', 'Y_train', 'Y_valid']>

In [9]:
conv_file.keys()

<KeysViewHDF5 ['X_test', 'X_train', 'X_valid', 'Y_test', 'Y_train', 'Y_valid']>

In [11]:
conv_file['X_test']

<HDF5 dataset "X_test": shape (1137, 173, 512), type "<f4">